In [32]:
"""
Author
Johan Tejning  - jote16@student.bth.se
Tokunbo Ojo    -  tooj21@student.bth.se
"""

'\nAuthor\nJohan Tejning  - jote16@student.bth.se\nTokunbo Ojo    -  tooj21@student.bth.se\n'

In [33]:
#Import libraries
from scipy import stats
from scipy.stats import studentized_range
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import time
import math

In [34]:
#Read spambase.data into pandas dataframe
df = pd.read_csv('spambase.data',header = None)
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [35]:
x = df.iloc[:, 0:57]
y = df[57]

In [36]:
# create discretizer for Equal Width Discretizer
kbins = KBinsDiscretizer(n_bins=5, strategy='uniform', encode='ordinal')

In [37]:
# Equal-Width Discretization implementation
def descretize(data):
    frames = []
    for i in range(0,57):
        bin = kbins.fit_transform(np.array(data.iloc[:, i:i + 1]).reshape(-1,1))
        frames.append(pd.DataFrame(bin, columns=[str(i)]))
    return pd.concat(frames, axis=1)
df = descretize(x)
df[57] = y
df[57]

0       1
1       1
2       1
3       1
4       1
       ..
4596    0
4597    0
4598    0
4599    0
4600    0
Name: 57, Length: 4601, dtype: int64

In [38]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
   # "Decision Tree",
   # "Random Forest",
    "Naive Bayes"
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
  #  DecisionTreeClassifier(max_depth=5),
  #  RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    GaussianNB()
]

In [39]:
# iterate over classifiers
performance = {"Nearest Neighbors" : [], "Linear SVM": [], "Naive Bayes": []}
accuracy = {"Nearest Neighbors" : [], "Linear SVM": [], "Naive Bayes": []}
f_measure = {"Nearest Neighbors" : [], "Linear SVM": [], "Naive Bayes": []}
fold = []

def train_model(train, test, fold_no):
    X_train = train.iloc[:, 0:57]
    y_train = train[57]
    X_test = test.iloc[:, 0:57]
    y_test = test[57]
    for name, clf in zip(names, classifiers):
        start = time.time()
        clf.fit(X_train, y_train)
        end = time.time()
        performance[name].append(end - start)
        y_pred = clf.predict(X_test) # make predictions on test set
        acc = accuracy_score(y_test.values, y_pred)
        accuracy[name].append(acc)
        f_score = f1_score(y_test.values, y_pred)
        f_measure[name].append(f_score)
    fold.append(fold_no)



In [40]:
# Initialise a StratifiedKFold object and declare the number of splits we want, which in our case is 10.
skf = StratifiedKFold(n_splits=10)
fold_no = 1

for train_index, test_index in skf.split(df, df[57]):
    train = df.loc[train_index,:]
    test = df.loc[test_index,:]
    train_model(train, test, fold_no)
    fold_no += 1


In [41]:
def calculate_rank(metrics):
    rank1 = []
    rank2 = []
    rank3 = []
    for index, row in metrics.iterrows():
        temp = row.copy()
        sorted = -np.sort(-temp.values)
        rank1.append(sorted.tolist().index(row[0]) + 1)
        rank2.append(sorted.tolist().index(row[1]) + 1)
        rank3.append(sorted.tolist().index(row[2]) + 1)

        result = {}
    result["Nearest Neighbors"] = rank1
    result["Linear SVM"] = rank2
    result["Naive Bayes"] = rank3
    ranking_df = pd.DataFrame.from_dict(result)
    return ranking_df

In [42]:
#1) computational performance in terms of training time,
training_time_metrics = pd.DataFrame(performance, index = fold)
training_time_metrics

,Nearest Neighbors,Linear SVM,Naive Bayes
1,0.007046,0.573260,0.004305
2,0.006795,0.528586,0.003283
3,0.006208,0.502713,0.003837
4,0.006921,0.521131,0.003158
5,0.006081,0.525922,0.003268
6,0.006971,0.566432,0.003206
7,0.005294,0.506429,0.003127
8,0.006242,0.509151,0.003066
9,0.006109,0.492861,0.003051
10,0.005303,0.504378,0.003083


In [43]:
#Calculate the ranks of training time metrics
training_time_metrics_ranking = calculate_rank(training_time_metrics)
training_time_metrics_ranking

,Nearest Neighbors,Linear SVM,Naive Bayes
0,2,1,3
1,2,1,3
2,2,1,3
3,2,1,3
4,2,1,3
5,2,1,3
6,2,1,3
7,2,1,3
8,2,1,3
9,2,1,3


In [44]:
#) predictive performance based on accuracy,
accuracy_metrics = pd.DataFrame(accuracy, index = fold)
accuracy_metrics

,Nearest Neighbors,Linear SVM,Naive Bayes
1,0.544469,0.676790,0.548807
2,0.571739,0.739130,0.567391
3,0.571739,0.682609,0.580435
4,0.595652,0.695652,0.569565
5,0.630435,0.747826,0.641304
6,0.571739,0.736957,0.565217
7,0.645652,0.769565,0.652174
8,0.552174,0.697826,0.567391
9,0.482609,0.663043,0.476087
10,0.486957,0.693478,0.484783


In [45]:
#Calculate the ranks of accuracy metrics
accuracy_metrics_ranking = calculate_rank(accuracy_metrics)
accuracy_metrics_ranking

,Nearest Neighbors,Linear SVM,Naive Bayes
0,3,1,2
1,2,1,3
2,3,1,2
3,2,1,3
4,3,1,2
5,2,1,3
6,3,1,2
7,3,1,2
8,2,1,3
9,2,1,3


In [46]:
#3) predictive performance based on F-measure.
f_measure_metrics = pd.DataFrame(f_measure, index = fold)
f_measure_metrics

,Nearest Neighbors,Linear SVM,Naive Bayes
1,0.620939,0.391837,0.628571
2,0.647585,0.562044,0.646536
3,0.646320,0.438462,0.652252
4,0.655556,0.440000,0.642599
5,0.674330,0.570370,0.683301
6,0.645045,0.566308,0.644128
7,0.688337,0.589147,0.692308
8,0.629496,0.441767,0.641441
9,0.595238,0.436364,0.596315
10,0.601351,0.442688,0.600337


In [47]:
#Calculate the ranks of f measure metrics
f_measure_metrics_ranking = calculate_rank(f_measure_metrics)
f_measure_metrics_ranking

,Nearest Neighbors,Linear SVM,Naive Bayes
0,2,3,1
1,1,3,2
2,2,3,1
3,1,3,2
4,2,3,1
5,1,3,2
6,2,3,1
7,2,3,1
8,2,3,1
9,1,3,2


In [48]:
rank_avg_training_time_metrics = training_time_metrics_ranking.mean(axis=0)
rank_avg_training_time_metrics

Nearest Neighbors    2.0
Linear SVM           1.0
Naive Bayes          3.0
dtype: float64

In [49]:
rank_avg_accuracy_metrics = accuracy_metrics_ranking.mean(axis=0)
rank_avg_accuracy_metrics

Nearest Neighbors    2.5
Linear SVM           1.0
Naive Bayes          2.5
dtype: float64

In [50]:
rank_avg_f_measure_metrics = f_measure_metrics_ranking.mean(axis=0)
rank_avg_f_measure_metrics

Nearest Neighbors    1.6
Linear SVM           3.0
Naive Bayes          1.4
dtype: float64

In [51]:
#Finding the sum of squared differences
def sum_of_squared_differences_1(rank_avg):
    sum = 0
    for x in rank_avg.values:
        sum = sum + (x - 2) ** 2
    sum_of_squared_differences_1 = 10 * sum
    return sum_of_squared_differences_1

In [52]:
#Finding the sum of squared differences 2
def sum_of_squared_differences_2(ranking_df):
    summation = 0
    for index, row in ranking_df.iterrows():
        summation = summation + ((row[0] - 2) ** 2) + ((row[1] - 2) ** 2) + ((row[2] - 2) ** 2)
    sum_of_squared_differences_2 = summation * 0.05
    return sum_of_squared_differences_2

In [53]:
#Finding p-value
p_value = scipy.stats.chi2.ppf(0.95, 3)
p_value

7.814727903251179

In [54]:
def friedman_statistic(rank_avg, ranking_df):
    friedman_statistic = sum_of_squared_differences_1(rank_avg) / sum_of_squared_differences_2(ranking_df)
    decision = "greater than " if friedman_statistic > p_value else "less than "
    decision2 = "can " if friedman_statistic > p_value else "can no "
    print("Since the Friedman statistic " + str(friedman_statistic) + " is " + decision + str(p_value) + " so we " + decision2 + "reject the null hypothesis that all algorithms perform equally")

In [55]:
# Training time metrics Friedman statistic
friedman_statistic(rank_avg_training_time_metrics, training_time_metrics_ranking)

Since the Friedman statistic 20.0 is greater than 7.814727903251179 so we can reject the null hypothesis that all algorithms perform equally


In [56]:
# Accuracy metrics Friedman statistic
friedman_statistic(rank_avg_accuracy_metrics, accuracy_metrics_ranking)

Since the Friedman statistic 15.0 is greater than 7.814727903251179 so we can reject the null hypothesis that all algorithms perform equally


In [57]:
# F Measure metrics Friedman statistic
friedman_statistic(rank_avg_f_measure_metrics, f_measure_metrics_ranking)

Since the Friedman statistic 15.2 is greater than 7.814727903251179 so we can reject the null hypothesis that all algorithms perform equally


In [58]:
#Finding critical difference (CD) using Nemenyi test
#The q alpha 2.343 is taken from the course textbook and also from this link https://plos.figshare.com/articles/dataset/Critical_values_for_the_two-tailed_Nemenyi_test_/7299293/1
#k = 3, n =  10

cd = 2.343 * math.sqrt(3 * (3 + 1) / (6 * 10))
cd

1.0478214542564015

In [59]:
#Nearest Neighbors - rank_avg[0]
#Linear SVM - rank_avg[1]
#Naive Bayes - rank_avg[2]
def critical_difference(rank_avg):
    no_of_means__of_ranks = len(rank_avg)
    for i in range(no_of_means__of_ranks - 1):
        for j in range(i + 1, no_of_means__of_ranks):
            diff = abs(rank_avg[i] - rank_avg[j])
            decision = " and it exceeds the critical difference " if diff > cd else " and does not exceeds the critical difference "
            print("The difference of " + rank_avg.index[i] + " - " + str(rank_avg[i])  + " and " + rank_avg.index[j] + " - " + str(rank_avg[j]) + " is " + str(diff) +  decision + str(cd))


In [60]:
# Training time metrics Nemenyi test
critical_difference(rank_avg_training_time_metrics)

The difference of Nearest Neighbors - 2.0 and Linear SVM - 1.0 is 1.0 and does not exceeds the critical difference 1.0478214542564015
The difference of Nearest Neighbors - 2.0 and Naive Bayes - 3.0 is 1.0 and does not exceeds the critical difference 1.0478214542564015
The difference of Linear SVM - 1.0 and Naive Bayes - 3.0 is 2.0 and it exceeds the critical difference 1.0478214542564015


In [61]:
# Accuracy metrics Nemenyi test
critical_difference(rank_avg_accuracy_metrics)

The difference of Nearest Neighbors - 2.5 and Linear SVM - 1.0 is 1.5 and it exceeds the critical difference 1.0478214542564015
The difference of Nearest Neighbors - 2.5 and Naive Bayes - 2.5 is 0.0 and does not exceeds the critical difference 1.0478214542564015
The difference of Linear SVM - 1.0 and Naive Bayes - 2.5 is 1.5 and it exceeds the critical difference 1.0478214542564015


In [62]:
# F Measure Nemenyi test
critical_difference(rank_avg_f_measure_metrics)

The difference of Nearest Neighbors - 1.6 and Linear SVM - 3.0 is 1.4 and it exceeds the critical difference 1.0478214542564015
The difference of Nearest Neighbors - 1.6 and Naive Bayes - 1.4 is 0.20000000000000018 and does not exceeds the critical difference 1.0478214542564015
The difference of Linear SVM - 3.0 and Naive Bayes - 1.4 is 1.6 and it exceeds the critical difference 1.0478214542564015
